In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])






cpu


In [10]:
#Data Loader
train_path='C:/Users/VIBISHNA/Downloads/noisedata/train'
test_path='C:/Users/VIBISHNA/Downloads/noisedata/test'
train_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),
                       batch_size=100,shuffle=True)
test_loader=DataLoader(torchvision.datasets.ImageFolder(test_path,transform=transformer),
                       batch_size=100,shuffle=True)
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1]for j in root.iterdir()])
print(classes)

['gaussian', 'poisson', 'random', 'saltandpepper', 'speckle']


In [11]:
#CNN
class ConvNet(nn.Module):
    def __init__(self,num_classes=8):
        super(ConvNet,self).__init__()
        #Output
        #(w-f+2P)/s)+1
        
        #Input Shape =(100,3,150,150)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        #Input Shape =(100,3,150,150)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #self.bn2=nn.BatchNorm2d(num_features=12)
        self.relu2=nn.ReLU()
        
        #Input Shape =(100,3,150,150)
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes)
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        output=output.view(-1,32*75*75)
        output=self.fc(output)
        return output

In [12]:
model=ConvNet(num_classes=8).to(device)
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
num_epochs=50

In [13]:
import glob

# Define the file extensions you want to include
allowed_extensions = ['jpg', 'jpeg', 'png', 'bmp', 'gif']  # Add any other formats you want

# Count training images
train_count = sum(len(glob.glob(train_path + f'/**/*.{ext}', recursive=True)) for ext in allowed_extensions)

# Count test images
test_count = sum(len(glob.glob(test_path + f'/**/*.{ext}', recursive=True)) for ext in allowed_extensions)

print("Total training images:", train_count)
print("Total test images:", test_count)

Total training images: 25
Total test images: 15


In [14]:
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    train_accuracy = train_accuracy / train_count

    model.eval()
    test_accuracy = 0.0
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
    test_accuracy = test_accuracy / test_count

    print('Epoch ' + str(epoch) + ' Train loss: ' + str(train_loss) + ' Train Accuracy ' + str(train_accuracy) + ' Test accuracy: ' + str(test_accuracy))
    
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best.model')
        best_accuracy = test_accuracy
print('Overall Test Accuracy: ' + str(best_accuracy))


Epoch 0 Train loss: tensor(56.5018) Train Accuracy 0.12 Test accuracy: 0.2
Epoch 1 Train loss: tensor(432.8927) Train Accuracy 0.28 Test accuracy: 0.2
Epoch 2 Train loss: tensor(552.6454) Train Accuracy 0.4 Test accuracy: 0.2
Epoch 3 Train loss: tensor(791.4326) Train Accuracy 0.32 Test accuracy: 0.26666666666666666
Epoch 4 Train loss: tensor(485.8346) Train Accuracy 0.4 Test accuracy: 0.26666666666666666
Epoch 5 Train loss: tensor(453.8187) Train Accuracy 0.24 Test accuracy: 0.3333333333333333
Epoch 6 Train loss: tensor(403.5342) Train Accuracy 0.44 Test accuracy: 0.4
Epoch 7 Train loss: tensor(326.8674) Train Accuracy 0.4 Test accuracy: 0.4
Epoch 8 Train loss: tensor(146.4740) Train Accuracy 0.52 Test accuracy: 0.3333333333333333
Epoch 9 Train loss: tensor(251.7201) Train Accuracy 0.52 Test accuracy: 0.2
Epoch 10 Train loss: tensor(135.9076) Train Accuracy 0.6 Test accuracy: 0.2
Epoch 11 Train loss: tensor(130.2379) Train Accuracy 0.56 Test accuracy: 0.26666666666666666
Epoch 12 Trai

In [15]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1  
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [16]:
pred_path='C:/Users/VIBISHNA/Downloads/noisedata/pred'

In [17]:
checkpoint=torch.load('best.model')
model=ConvNet(num_classes=8)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=8, bias=True)
)

In [18]:
def prediction(img_path, transformer):
    image = Image.open(img_path)
    
    # Check and convert to RGB if image has 4 channels
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    
    image_tensor = transformer(image).float()
    
    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()
    
    # Add a batch size dimension
    image_tensor = image_tensor.unsqueeze(0)
    
    input = Variable(image_tensor)
    output = model(input)
    index = output.data.numpy().argmax()
    pred = classes[index]
    return pred


In [19]:
import glob

# Define the allowed image extensions
allowed_extensions = ['jpg', 'jpeg', 'png', 'bmp', 'gif']  # Add any other formats you want

# Use glob to find all image files with allowed extensions
images_path = [file for ext in allowed_extensions for file in glob.glob(pred_path + f'/*.{ext}')]

# Now, images_path contains the paths of all image files with allowed extensions in pred_path


In [20]:
images_path

['C:/Users/VIBISHNA/Downloads/noisedata/pred\\g1.jpg',
 'C:/Users/VIBISHNA/Downloads/noisedata/pred\\image9.jpg',
 'C:/Users/VIBISHNA/Downloads/noisedata/pred\\r3.jpg',
 'C:/Users/VIBISHNA/Downloads/noisedata/pred\\sp2.jpg',
 'C:/Users/VIBISHNA/Downloads/noisedata/pred\\spe1.jpg']

In [21]:
pred_dict={}
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [22]:
pred_dict

{'pred\\g1.jpg': 'gaussian',
 'pred\\image9.jpg': 'poisson',
 'pred\\r3.jpg': 'random',
 'pred\\sp2.jpg': 'saltandpepper',
 'pred\\spe1.jpg': 'speckle'}